In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
# hyperparameter
lr = 0.01
gamma = 0.8
bs = 64
epochs = 10

In [3]:
# load data
mnist = torchvision.datasets.FashionMNIST(root = './Datastes', train = True, download = True, transform = transforms.ToTensor())
# build dataset
batchdata = DataLoader(mnist, batch_size = bs, shuffle = True, drop_last = False)

In [11]:
len(batchdata)

938

In [4]:
for x, y in batchdata:
    print(x.shape)
    print(y.shape)
    break
# torch.Size([128, 1, 28, 28])
# torch.Size([128])

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
# input dimension
input_ = mnist.data[0].numel()  # 784
# output dimension
output_ = len(mnist.targets.unique()) # 10

In [6]:
# model build
class Model(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features, 128, bias = True)
        self.output = nn.Linear(128, out_features, bias = True)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        sigma1 = torch.relu(self.linear1(x))
        sigma2 = F.log_softmax(self.output(sigma1), dim = -1)
        return sigma2

In [22]:
# fit model
def train_fit(net, batchdata, lr, gamma, epochs):
    # loss function
    criterion = nn.NLLLoss()
    # optimizer
    opt = optim.SGD(net.parameters(), lr = lr, momentum = gamma)
    
    # samples back to square one
    samples = 0
    # corrects back to square one
    corrects = 0
    
    # training epoch
    for epoch in range(epochs):
        for batch_idx, (x, y) in enumerate(batchdata):
            # label to one dimension
            y = y.view(x.shape[0])
            
            # forward
            sigma = net.forward(x)
            # calculate loss 
            loss = criterion(sigma, y)
            # backward
            loss.backward()
            # grad()
            opt.step()
            # grad back to square one
            opt.zero_grad()
            
            samples += x.shape[0]
            
            yhat = torch.max(sigma, -1)[1]
            corrects += torch.sum(yhat == y)
            
            # print model progress rate
        print("Epoch{}: Loss:{:.6f}, Accuracy:{:.6f}".format(epoch + 1, loss.data.item(),float(100.0*corrects/samples)))

In [23]:
# set random seed
torch.manual_seed(51)

net = Model(input_, output_)

# training model  , finally
train_fit(net, batchdata, lr, gamma, epochs)

Epoch1: Loss:0.643133, Accuracy:75.908333
Epoch2: Loss:0.744018, Accuracy:79.631668
Epoch3: Loss:0.517258, Accuracy:81.338890
Epoch4: Loss:0.579227, Accuracy:82.381668
Epoch5: Loss:0.319061, Accuracy:83.153000
Epoch6: Loss:0.245719, Accuracy:83.772499
Epoch7: Loss:0.376630, Accuracy:84.286194
Epoch8: Loss:0.600894, Accuracy:84.707291
Epoch9: Loss:0.349639, Accuracy:85.072777
Epoch10: Loss:0.249381, Accuracy:85.407166
